In [12]:
import pandas as pd
import numpy as np
import os
from pprint import pprint

In [14]:
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/Voted/'
files = os.listdir(DATA_DIR)
files = [i for i in files if i[0]!='.']
print(len(files))
pprint(files)

6
['submission_bert4patent.csv',
 'submission_bert Large.csv',
 'submission _mengzi.csv',
 'submission_bert4p_BTAug.csv',
 'submission_bert4p_sp.csv',
 'submission_Mixed.csv']


In [15]:
#train = pd.read_csv('./data/train.csv', sep='\t')

df_merged = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/submission.csv')
df_merged = df_merged.drop(['label'], axis=1)
df_merged.head()

,id
0,003fd481e65ddc070e38ae05002e16e2
1,549a1cd8228bd10f18395a0625fcc70d
2,f09c4c0332f8966400e06f4def9f1a6d
3,06598dd8f3ab092acf2a55dce8be5621
4,e70177ba6a54d08abecd80a60fdd9f52


In [16]:
for file in files:
    tmp_df = pd.read_csv(DATA_DIR + file)
    df_merged = df_merged.merge(tmp_df, how='left', on='id')
df_merged.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'label_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


,id,label_x,label_y,label_x,label_y,label_x,label_y
0,003fd481e65ddc070e38ae05002e16e2,23,23,23,23,23,23
1,549a1cd8228bd10f18395a0625fcc70d,5,5,5,5,5,5
2,f09c4c0332f8966400e06f4def9f1a6d,5,5,5,5,5,5
3,06598dd8f3ab092acf2a55dce8be5621,16,16,16,16,16,16
4,e70177ba6a54d08abecd80a60fdd9f52,0,0,0,0,0,0


In [18]:
def work_high(pres):
    count = [0]*36
    for i in pres:
        count[i] += 1
    p = 5 # 可根据融合模型的数量选择该数的大小。当共有8个模型融合，>7表示必须有8个模型都投票的才被选择加入伪标签数据
    if max(count) >p:
        out = count.index(max(count))
    else:
        out = -1
    return out

tmp_arr = np.array(df_merged.iloc[:,1:])
label_voted = [work_high(line) for line in tmp_arr]
print(label_voted.count(-1))

7870


In [19]:
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/testA_zh_en_final.csv')
test_data['label_id'] = label_voted
test_data = test_data.drop(test_data[test_data['label_id']==-1].index)
len(test_data)

12969

In [20]:
test_data.head()

,id,title,assignee,abstract,title_en,assignee_en,abstract_en,label_id
0,003fd481e65ddc070e38ae05002e16e2,一种耐磨、抗粘钢复合涂层、制备方法及应用,安徽马钢表面技术股份有限公司,本发明公开了一种耐磨、抗粘钢复合涂层、制备方法及应用，包括基体和基体上由内到外依次设置的过渡...,Wear-resistant and anti-adhesive steel composi...,"Anhui Maanshan Steel Surface Technology Co., Ltd.",The invention discloses a wear-resistant and a...,23
1,549a1cd8228bd10f18395a0625fcc70d,一种用于提高橡胶抗湿滑性的树脂的制备方法及其应用,江苏麒祥高新材料有限公司,本发明公开了一种用于提高橡胶抗湿滑性的树脂的制备方法，第一步：将R树脂和B官能团化合物进行反...,Preparation method and application of resin fo...,"Jiangsu Qixiang High-tech material Co., Ltd.",The invention discloses a method for preparing...,5
2,f09c4c0332f8966400e06f4def9f1a6d,有机硅改性丙烯酸树脂超亲水防雾涂料及其制作方法,重庆大学,本发明涉及涂料制造领域，本发明公开了一种含有磺酸季铵盐的有机硅改性丙烯酸树脂超亲水低温防雾涂...,Silicone modified acrylic resin super hydrophi...,Chongqing University,The invention relates to the field of paint ma...,5
3,06598dd8f3ab092acf2a55dce8be5621,一种空调系统及其控制方法、控制装置,海尔智家股份有限公司,本发明涉及空调领域，公开了一种空调系统，包括室外机和太阳能供热系统，所述太阳能供热系统包括：...,"Air conditioning system, control method and co...","Haier Zhijia co., Ltd.",The invention relates to the air conditioning ...,16
4,e70177ba6a54d08abecd80a60fdd9f52,资源申请、分配方法，UE及网络控制单元,中兴通讯股份有限公司,本发明实施例公开了一种资源申请方法及装置，所述方法包括：向网络控制单元发送低时延业务信息；接...,"Resource application, allocation method, UE an...","Chung Hsing communication co., Ltd.",The embodiment of the invention discloses a re...,0


In [21]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/train_en.csv',index_col=0)
train.head()

,id,title,assignee,abstract,title_en,assignee_en,abstract_en,label_id
0,538f267d2e6fba48b1286fb7f1499fe7,一种信号的发送方法及基站、用户设备,华为技术有限公司,一种信号的发送方法及基站、用户设备。在一个子帧中为多个用户设备配置的参考信号的符号和数据的符...,"Signal transmission method, base station and u...",Huawei Technology Co. Ltd.,The invention relates to a method for transmit...,0
1,635a7d4b6358b6ff24a324bb871505db,一种5G通讯电缆故障监控系统,中铁二十二局集团电气化工程有限公司,本发明公开了一种5G通讯电缆故障监控系统，包括信号采样模块、补偿反馈模块，所述信号采样模块对...,A Fault Monitoring system for 5G Communication...,China Railway 22 Bureau Group electrification ...,The invention discloses a 5G communication cab...,0
2,aaf98d6bfe1932cf1a262812ca59d1ba,一种测试方法及电子设备,腾讯科技(北京)有限公司,本发明提供了一种测试方法及电子设备，该方法包括：基于选取的测试任务确定目标测试用例，根据所述...,Test method and electronic device,"Tencent Technology (Beijing) Co., Ltd.",The invention provides a test method and an el...,0
3,ad49c9ba6412452d9b25071af702f4fb,天线方位角调节装置,武汉虹信通信技术有限责任公司,一种天线方位角调节装置，包括对向的两个8字形支架(101)、动力输入电机(102)、主动齿轮...,Antenna azimuth adjusting device,"Wuhan Hongxin Communication Technology Co., Ltd.",An antenna azimuth adjusting device comprises ...,0
4,ffa2d7617b3eac3a1d7df01e5bb515a2,光纤老化预测方法及装置,新华三大数据技术有限公司,本申请提供一种光纤老化预测方法及装置，所述方法包括：获取待测光纤模块可接收的光信号的告警阈值...,Optical fiber aging prediction method and device,"Xinhua San big data Technology Co., Ltd.",The application provides an optical fiber agin...,0


In [22]:
pseudo_label_train = pd.concat([train,test_data])
pseudo_label_train.to_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/aug_data/pseudo_label_train_6models_without_mengzi.csv', index=False)